In [2]:
# !wget --header="Host: doc-0s-54-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Cookie: AUTH_prqi0ddebcnouinrgq4n8k579qees3ri=09250325709264443050|1674244950000|1c1ogi62gqa0o073nvbr2kl2f6fikp72" --header="Connection: keep-alive" "https://doc-0s-54-docs.googleusercontent.com/docs/securesc/5n63bj25lj97nngplfo91cqd1qhl5kn0/8ej607icfh0m6j3fb5tj6udnh0la5itf/1674245025000/06977169721700109230/09250325709264443050/1z2lFY4SWkYU82vMtX1nYaMQZwU0a4HvR?e=download&ax=ALjR8sxbmW2AJUtlvX8dw48uRbOUdNPAYG91QPtqFSN38R7BL3ClSGdI95QBmCoOWDZBbub-fCLCrKdUtwrWWqTJjWzsCw28oxWDeF1ad7L3L-Dn_NpnD7EVG8hRCVjjyHhg5kczrEjIhUGJFgkCTXSdFPLALYTFvRHD7GEWxJul0QXLUm2hrMJjIhCQKkmLX39B6iO-HTEXRbw0OgPLZqkpMVwZyrtpWecRAir_yYqkulr2sPZ21TS30XoazMqRU7yfsKfqLLeMmGyc4O6Qa1bxBhp4nhwE6EjlenB1MDnN3zEPMctoUq6tCBvezYC6nQme-VIbhTdLDhKvBHp_VcDZ23KHDp_UlvPMPQkIBmXS_ihZRHyv2c1N_Zq5SJ2Z8iRJuNQoFYMTQbZNQA657HEa30uIupKQVDBPmb1k1Hgm5lkPfUP9zaWKWd77i4fd7ZyG1fnRamXiHYyHV2Kb9XXJt1KvBgKbv72aX_qDWwbRF1Xiy_lmGMYQ68GE0pRSRmhDnu0j8wllqYS7gbHuWuNqRFyHz_d4otycw0kMvVSV_dQkHcU04y1_tCIwBfL2xZixJJ-0qIE8ns-sTCLDPb0C_tNxufUDrQoYohqdyOHLfpLPrH9TDgtcUahBRj7TpFrP4MwuaILpYF-Pjbj5ifg1e8xyjdFODzlHTg0RWeqeaGWjGIh22OidVACNEFPwCClrZOBU7qx3W5Sn8VSBBWFLtSC3mGSoKCaryqmiGmeoqTwaAwz5_2rhaB-s-NjdLDXpc88cyqNhVsgueMVfFfdf_BK1shiqV_UGlsg1teXgPXeMSm03HxU1kWaT01_Qnm7lRT8WdZNXd8ndFObio_BAY5y8CEydet7gsoRd4cHTIuqs7EIB02faHSJIKkvVBqXY3ZtPGgkc3hjKGShU1ta-UfS2hEE4Sx_9NCjBYvoEJHuESI_l8PaSFY839dS8jKc0oM0umPPLS4SgkX4ORyL4DtkYmHVv0luOMSc&uuid=66b0f1f1-2bd2-4c63-8fba-60123d3f2d1c&authuser=2" -c -O 'csebuetnlp-banglabert_large.h5'

In [3]:
import numpy as np
import pandas as pd

!pip install transformers
from transformers import AutoTokenizer

from tqdm import tqdm
import tensorflow as tf

import matplotlib.pyplot as plt

Keyring is skipped due to an exception: 'keyring.backends'


In [4]:
def format(file):
  with open(file) as f:
      # read csv file as pandas dataframe
      df = pd.read_csv(f, sep=',', 
                       # names=['Sentence_id', 'Word', 'Tag']
                      )
      # group by sentence_id and create list of words and list of tags
      df = df.groupby('sentence_id').agg({'word': lambda x: list(x), 'tag': lambda x: list(x)})
      # create pair of 2nd and 3rd column (word , tag)
      df['word_tag'] = df.apply(lambda x: list(zip(x['word'], x['tag'])), axis=1)
      # create list of pair of 2nd and 3rd column (word , tag)
      word_tag_list = df['word_tag'].tolist()
      
      return word_tag_list

file = 'train_generic.csv'
samples = format(file)

In [5]:
print(samples[:5])

[[('তার', 'O'), ('মৃত্যুর', 'O'), ('দশ', 'O'), ('দিন', 'O'), ('পর,', 'O'), ('১১৫', 'O'), ('কৃষ্ণাঙ্গ', 'O'), ('উচ্চ', 'O'), ('বিদ্যালয়ের', 'O'), ('শিক্ষার্থীরা', 'O'), ('তার', 'O'), ('হত্যার', 'O'), ('প্রতিবাদে', 'O'), ('ম্যাককম্ব', 'B-LOC'), ('এর', 'O'), ('মাধ্যমে', 'O'), ('মিছিল', 'O'), ('করেছে।', 'O')], [('ব্রাংম্যান', 'O'), ('ডাম্পসন', 'O'), ('১৪০০', 'O'), ('সালে', 'O'), ('আন্তর্জাতিক', 'B-GRP'), ('রেড', 'I-GRP'), ('ক্রস', 'I-GRP'), ('ও', 'I-GRP'), ('রেড', 'I-GRP'), ('ক্রিসেন্ট', 'I-GRP'), ('আন্দোলন', 'I-GRP'), ('এর', 'O'), ('স্বেচ্ছাসেবক', 'O'), ('হিসেবে', 'O'), ('শুরু', 'O'), ('করেছিলেন।', 'O')], [('রাজকীয়', 'O'), ('বাসস্থান', 'O'), ('থেকে', 'O'), ('রাজ্যের', 'O'), ('মন্দির', 'O'), ('পর্যন্ত', 'O'), ('স্বল্প', 'O'), ('দূরত্ব', 'O'), ('ভ্রমণ', 'O'), ('করে,', 'O'), ('রাজা', 'O'), ('মিছিলে', 'O'), ('অনেক', 'O'), ('ধারক', 'O'), ('নিয়ে', 'O'), ('পালকি', 'B-PROD'), ('বসবেন।', 'O')], [('তিনি', 'O'), ('তৃতীয়', 'O'), ('সহস্রাব্দে', 'O'), ('গ্র্যান্ড', 'B-CW'), ('ওলে', 'I-CW'), ('অপ্রি

In [6]:
schema = ['_'] + sorted({tag for sentence in samples 
                             for _, tag in sentence})

In [7]:
print(schema)

['_', 'B-CORP', 'B-CW', 'B-GRP', 'B-LOC', 'B-PER', 'B-PROD', 'I-CORP', 'I-CW', 'I-GRP', 'I-LOC', 'I-PER', 'I-PROD', 'O']


In [8]:
from transformers import AutoConfig, TFAutoModelForTokenClassification , AutoTokenizer

#MODEL_NAME = 'bert-base-cased' 
# MODEL_NAME = "csebuetnlp/banglabert"
MODEL_NAME = "csebuetnlp/banglabert_large"


config = AutoConfig.from_pretrained(MODEL_NAME, num_labels=len(schema)) 

#old_model = 'csebuetnlp-banglabert0.6987703481430697.h5'
old_model = 'data/csebuetnlp-banglabert_large_f1_81.h5'
model = TFAutoModelForTokenClassification.from_pretrained(old_model, from_pt=False,
                                                       config=config)
                                          
optimizer = tf.keras.optimizers.Adam(lr=0.000001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, 
              metrics=["accuracy"])

model.summary()

All model checkpoint layers were used when initializing TFElectraForTokenClassification.

All the layers of TFElectraForTokenClassification were initialized from the model checkpoint at data/csebuetnlp-banglabert_large_f1_81.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraForTokenClassification for predictions without further training.


Model: "tf_electra_for_token_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 electra (TFElectraMainLayer  multiple                 335605760 
 )                                                               
                                                                 
 dropout_73 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  14350     
                                                                 
Total params: 335,620,110
Trainable params: 335,620,110
Non-trainable params: 0
_________________________________________________________________


/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [11]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def format_test(file):
  with open(file) as f:
      # read csv file as pandas dataframe
      df = pd.read_csv(f, sep=',', 
                       # names=['Sentence_id', 'Word', 'Tag']
                      )
      # group by sentence_id and create list of words and list of tags
      df = df.groupby('sentence_id').agg({'word': lambda x: list(x)})
      # create pair of 2nd and 3rd column (word , tag)
      # df['word_tag'] = df.apply(lambda x: list(zip(x['word'], x['tag'])), axis=1)
      # create list of pair of 2nd and 3rd column (word , tag)
      word_list = df['word'].tolist()
      
      return word_list
subtoken_map = {}
def tokenize_sample(sample):
    # seq = [
    #            (subtoken, tag)
    #            for token, tag in sample
    #            for subtoken in tokenizer(token)['input_ids'][1:-1]
    #        ]
    seq = []
    sub_lengths = []
               # (subtoken, tag)
    for token in sample:
        subtokens = tokenizer(token)['input_ids'][1:-1]
        # subtoken_map
        for subtoken in subtokens:
            seq.append((subtoken))
        sub_lengths.append(len(subtokens))
           
    return [(3)] + seq + [(4)], sub_lengths 

def preprocess(samples):
    # tag_index = {tag: i for i, tag in enumerate(schema)}
    # tokenized_samples = list(tqdm(map(tokenize_sample, samples)))
    tokenized_samples = []
    all_sub_lengths = []
    for s in tqdm(samples):
        seq, lengths = tokenize_sample(s)
        tokenized_samples.append(seq)
        all_sub_lengths.append(lengths)
        
        # print(lengths, sum(lengths))
        # break
        
    max_len = max(map(len, tokenized_samples))
    print(max_len)
    X = np.zeros((len(samples), max_len), dtype=np.int32)
    for i, sentence in enumerate(tokenized_samples):
        for j, (subtoken_id) in enumerate(sentence):
            X[i, j] = subtoken_id
        
    return X, all_sub_lengths

# file = 'sample_test_set_no_punc.csv'
file = ''
dev_samples = format_test(file)
X_valid, all_sub_lengths = preprocess(dev_samples)

100%|██████████| 2/2 [00:00<00:00, 743.67it/s]

27


In [12]:
# [loss, accuracy] = model.evaluate(X_valid, y_valid)
# print("Loss:%1.3f, Accuracy:%1.3f" % (loss, accuracy))

# Find the f1 score of the model output
from sklearn.metrics import f1_score
y_pred = model.predict(X_valid).logits
y_pred_raw = np.argmax(y_pred, axis=-1)
print(type(y_pred_raw))


1/1 [==============================] - 9s 9s/step
<class 'numpy.ndarray'>


In [13]:
y_pred_raw.shape

(2, 27)

In [14]:
reverse_tag_index = {i:tag for i, tag in enumerate(schema)}
print(reverse_tag_index)
lens = []
for s in dev_samples:
    lens.append(len(s))
    
# print(lens)
print(max(lens))
cum_len = list(np.cumsum(lens))
# print(cum_len)

dev_pred_out = open('dev_pred_ours.txt', 'w')
# y_pred = list(filter(lambda a: a!= 0, y_pred) )
# print(y_pred[0])

labels = []

for i in range(y_pred_raw.shape[0]):
    # print(y , reverse_tag_index[y])
    index = 1
    for sublen in all_sub_lengths[i]:
        # print(reverse_tag_index[y_pred_raw[i][index]], file=dev_pred_out)
        label = reverse_tag_index[y_pred_raw[i][index]]
        labels.append(label)
        
        index += sublen
        
    # for j in range(1, lens[i] + 1):
    #     # print(y_pred_raw[i, j])
    #     print(reverse_tag_index[y_pred_raw[i][j]], file=dev_pred_out)
        
    # if (i + 1) in cum_len:
    labels.append('')
    
    # print(file=dev_pred_out)
    # break

dev_pred_out.close()

{0: '_', 1: 'B-CORP', 2: 'B-CW', 3: 'B-GRP', 4: 'B-LOC', 5: 'B-PER', 6: 'B-PROD', 7: 'I-CORP', 8: 'I-CW', 9: 'I-GRP', 10: 'I-LOC', 11: 'I-PER', 12: 'I-PROD', 13: 'O'}
15


In [15]:
# for i, label in enumerate(labels):
#     if i + 1 < len(labels):
#         if len(labels[i]) > 0 and len(labels[i+1]) > 0:
#             if labels[i][0] == "B" and labels[i+1][0] == "B":
#                 labels[i+1][0] == "I"

dev_pred_out = open('dev_pred_ours.txt', 'w')
for i, label in enumerate(labels):
    print(label, file=dev_pred_out)

dev_pred_out.close()                                     

In [71]:
# !python eval_script/eval_main.py --true eval_script/dev_gt_labels.txt --pred dev_pred_ours.txt

{
  "P@GRP": 0.7641509433962257,
  "R@GRP": 0.6864406779661011,
  "F1@GRP": 0.7232142857142353,
  "P@LOC": 0.8315789473684202,
  "R@LOC": 0.7821782178217814,
  "F1@LOC": 0.8061224489795411,
  "P@CW": 0.6929824561403503,
  "R@CW": 0.6583333333333328,
  "F1@CW": 0.6752136752136247,
  "P@PROD": 0.7134502923976603,
  "R@PROD": 0.6421052631578944,
  "F1@PROD": 0.67590027700826,
  "P@PER": 0.8815789473684204,
  "R@PER": 0.9305555555555548,
  "F1@PER": 0.9054054054053546,
  "P@CORP": 0.7122302158273376,
  "R@CORP": 0.7795275590551175,
  "F1@CORP": 0.7443609022555886,
  "Precision": 0.7644787644787644,
  "Recall": 0.7424999999999999,
  "F1": 0.7533291058972232,
  "MD@R": 0.8425,
  "MD@P": 0.8674388674388674,
  "MD@F1": 0.8547875713379335,
  "ALLTRUE": 800,
  "ALLRECALLED": 674,
  "ALLPRED": 777
}
Precision 0.76, Recall 0.74, F1 0.75
